In [ ]:
## Agile Hardware Design
***
# Chisel Bag

<img src="./images/chisel_logo.svg" alt="agile hardware design logo" style="width: 20%; float:right"/>

Peter Hanping Chen, based on

- 1. UCB Bootcamp: configuration file load-ivy.sc: 
- https://github.com/freechipsproject/chisel-bootcamp/tree/master/source
- 2. Prof. Scott Beamer, sbeamer@ucsc.edu, CSE 228A
- https://classes.soe.ucsc.edu/cse228a/Winter24/

## Plan for Today

* Recap how Chisel "works"
* Tips for common mistakes to avoid

## Loading The Chisel Library Into a Notebook

In [16]:
// interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
//val path = System.getProperty("user.dir") + "/source/chisel_deps.sc"
println("path: "+path)
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/020_Misc/001_Code/source/load-ivy.sc


path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/020_Misc/001_Code/source/load-ivy.sc"

In [17]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

import chisel3._

import chisel3.util._

import chiseltest._

import chiseltest.RawTester.test

## Recap of What Chisel/Scala Does

* Your "Chisel" design is a valid Scala program (otherwise will get a Scala compile error)
  * Chisel is a Scala library, and we treat it like an _embedded domain-specific language_
* While executing, any Chisel object referenced/constructed is instantiated
  * Includes even a literal (e.g. `4.U`) or a slight tweak (e.g. `~io.in`)
  * Every object has _inputs_ and/or _outputs_
  * Under the hood, those objects are tracked (e.g. inside `extends Module`)
* Chisel connections (`:=` and `<>`) induce side-effects on the Chisel objects
  * Changes inputs by connecting them to outputs
* _Summary:_ think of your Chisel design as a Scala program that _instantiates_ Chisel things and _connects_ them
  * Much of the parameterizability/flexibility is coming from the Scala program
* Your goal is to properly connect things to the inputs/outputs of modules
  * Tools will prune components that are unreachable from an input or output

## Hardware Designs Are _Static Structurally_ with _Dynamic Signals_

* Hardware's connectivity/structure is _static_ after _elaboration_
  * Even though we are only simulating and not manufacturing the physical designs, the design is unchanged
  * A mux input can change its output, but it has static connections externally
* A wire can carry different values in different cycles (in simulation or real world), but the wire's endpoint connections are unchanged
  * A wire has no internal state, and directly propagates its input to its output
  * A wire changing value over "time" is caused by its input changing over time
* A _register_ (or memory) has internal state, but only changes value at the rising clock edge
  * At rising edge, input value becomes output (and internal state) value
  * Typically in Chisel we don't explicitly show clock, so sometimes easy to loose track of when things change

## Even with _Last Connect Semantics_, Hardware Structure is Static

* When there are multiple connections to the same input, Chisel must choose a winner
  * The actual hardware can only be connected to one thing
* _Last Connect Semantics_ - last connect performed in Scala program order "wins"
* `when` statements are handled specially with muxes
  * What to connect to depends on when's condition => use mux
  * Mux output is the one thing hardware is connected to
  * The when condition is used for mux select

In [18]:
class Clipper extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(8.W))
        val out = Output(UInt(8.W))
    })
    io.out := io.in
    when (io.in > 3.U) {
        io.out := 3.U
    }
}

//printVerilog(new Clipper)

defined class Clipper

In [19]:
//printVerilog(new Clipper)
println (getVerilog(new Clipper))

Elaborating design...
Done elaborating.
module Clipper(
  input        clock,
  input        reset,
  input  [7:0] io_in,
  output [7:0] io_out
);
  assign io_out = io_in > 8'h3 ? 8'h3 : io_in; // @[cmd17.sc 7:24 cmd17.sc 8:16 cmd17.sc 6:12]
endmodule



# Easy Simplification: Convert Nested Whens -> AND



In [20]:
class NestedWhens() extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(3.W))
        val out = Output(Bool())
    })
    io.out := false.B
    when (io.in(0)) {
        when (io.in(1)) {
            when (io.in(2)) {
                io.out := true.B
            }
        }
    }
}

//printVerilog(new NestedWhens)

defined class NestedWhens

In [21]:
//printVerilog(new NestedWhens)
println (getVerilog(new NestedWhens))

Elaborating design...
Done elaborating.
module NestedWhens(
  input        clock,
  input        reset,
  input  [2:0] io_in,
  output       io_out
);
  wire  _GEN_1 = io_in[1] & io_in[2]; // @[cmd19.sc 8:25 cmd19.sc 6:12]
  assign io_out = io_in[0] & _GEN_1; // @[cmd19.sc 7:21 cmd19.sc 6:12]
endmodule



## Mutability, but When & Where?

* _Mutability_ (e.g. `var` in Scala) impacts how your Scala program will behave
  * Chisel tools mostly can't tell if you declare things with `val` or `var`
  * Do not confuse mutability in Scala with hardware signal values varying in time
* Example of implementing a counter
    * **Incorrect** (`counter` (in elaborated hardware) will always be `0 + 1`)
    ```scala
    var counter = 0.U
    counter = counter + 1.U
    ```

    * **Better** (will increment over time, but may need to worry about reset & bitwidth)
    ```scala
    val counter = Reg(UInt())
    counter := counter + 1.U
    ```

## More Reason to Avoid `var`
* Reassignments can do weird things to your Chisel design that make it hard to debug 
* Chisel tools can't outright stop use of `var`, but progress has been made in detection/warning
  * ```Source has escaped the scope of the when in which it was constructed.```
* We have now covered `map`,`reduce`, and others, so not much need

In [22]:
class DangerousVar extends Module {
    val io = IO(new Bundle {
        val in = Input(SInt(8.W))
        val out = Output(SInt(8.W))
    })
    var w = WireInit(io.in)
    when (io.in < 0.S) {
        w := 0.S   // what if typo: w = 0.S
    }
    io.out := w
}

//printVerilog(new DangerousVar)

defined class DangerousVar

In [23]:
//printVerilog(new DangerousVar)
println (getVerilog(new DangerousVar))

Elaborating design...
Done elaborating.
module DangerousVar(
  input        clock,
  input        reset,
  input  [7:0] io_in,
  output [7:0] io_out
);
  assign io_out = $signed(io_in) < 8'sh0 ? $signed(8'sh0) : $signed(io_in); // @[cmd21.sc 7:24 cmd21.sc 8:11]
endmodule



## Use Mutable Collections Sparingly

* We used them for HW3 because we hadn't covered functional programming yet
* Now with functional programming, can often avoid the need
  * If things are independent, can use `map`, `foreach` (for Chisel connections), or `.tabulate` (to populate a `Seq`)
  * If some sort of loop-carried dependence, can use `foldLeft` or recursion
* Example of incrementing all values
    * **Gross**
        ```scala
        val a = ArrayBuffer.tabulate(5)(_.toInt)
        for (i <- 0 until 5)
          a(i) += 1
        ```
        * Uses mutation and iteration is a distraction
    * **Better**
        ```scala
        val orig = Seq.tabulate(5)(_.toInt)
        val incremented = orig.map{ _+1 }
        ```
        * No mutation or unnecessary iteration



## Reduce Number of Special Cases

* Saw some submissions which hardcoded cases for every parameter value anticipated
  * Not scalable if number of parameter values is large
* In general, look to reduce special cases in code
  * Many constructs (e.g. `foldLeft`) work gracefully with 0 elements, so just need to handle _sufficiently general_ case
  * If you do need to handle a special case, try to limit it to only one
    * If you need more, see if you can't _generalize_

## How Does `for` or `foreach` Interact with Chisel?

* Both `for` and `foreach` impact Scala execution when constructing the Chisel design
* Typically they are best used for creating arbitrary number of connections
  * The benefit is the side effect of connection(s) being made
  * If a result is the goal, then `map` is probably a better fit
* Style wise typically prefer...
  * `foreach` if collection/range already exists
  * `for` if creating range or need index variable
    * Yes, could use `.zipWithIndex` with `foreach`, but cumbersome
  

## Avoid Unnecessary Extra Logic

* Although CAD tools can often optimize away inefficient logic, still make modest effort
  * Simpler logic can also be simpler to read/maintain
* For working with 2D grids, we often saw `%` and `/` to pull out row & column indices from a single counter
  * Consider using 2 counters (1 for row & 1 for column)
  * `%` and `/` are quite expensive in hardware, so avoid if possible
  * Also saw `*` for generating single index, but may also be avoidable
* For accessing bits or moving bits, saw `<<` and `&` (with masks)
  * Necessary in software, but not in Chisel
  * Chisel has bit select `x(hi,lo)`, `tail` and `head` to select
  * Chisel has `Cat` to put them together

## Chisel Style - Avoid Declaring Significant HW inside `when`

* Hardware declared inside a `when` block is always instantiated/exists
  * The conditional aspects of `when` only control when connections to it are active (via muxes)
  * By contrast, a declaration inside a Scala `if` may not be instantiated
* Arguably, sometimes more clear to instantiate things outside to clarify intent

In [24]:
class CounterWhenDemo extends Module {
    val io = IO(new Bundle {
        val en = Input(Bool())
        val in = Input(UInt(8.W))
        val out = Output(UInt(8.W))
    })
    io.out := 0.U
    when (io.en) {
        val (count, wrap) = Counter(0 until 4)
        io.out := count
    }
}

//printVerilog(new CounterWhenDemo)

defined class CounterWhenDemo

In [25]:
//printVerilog(new CounterWhenDemo)
println (getVerilog(new CounterWhenDemo))

Elaborating design...
Done elaborating.
module CounterWhenDemo(
  input        clock,
  input        reset,
  input        io_en,
  input  [7:0] io_in,
  output [7:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [1:0] value; // @[Counter.scala 60:40]
  wire [1:0] _wrap_value_T_1 = value + 2'h1; // @[Counter.scala 76:24]
  wire [1:0] _GEN_4 = io_en ? value : 2'h0; // @[cmd23.sc 8:18 cmd23.sc 10:16 cmd23.sc 7:12]
  assign io_out = {{6'd0}, _GEN_4}; // @[cmd23.sc 8:18 cmd23.sc 10:16 cmd23.sc 7:12]
  always @(posedge clock) begin
    if (reset) begin // @[Counter.scala 60:40]
      value <= 2'h0; // @[Counter.scala 60:40]
    end else begin
      value <= _wrap_value_T_1;
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef

## Scala Style - Use either `until` or `to`

* Scala provides both `until` (exclusive bounds) and `to` (inclusive bounds)
  * Thus, usually shouldn't need to have `n-1` or `n+1` in a bound
* Can convert `0 to n-1` to `0 until n`

In [26]:
val n = 4
(0 until n) foreach println

0
1
2
3


n: Int = 4

## When to use `require` vs `assert`?

* Part of confusion is which `assert` (Chisel or Scala) gets called?
* Chisel `assert` checks value in _simulation,_ but not during _construction_
  * Emits non-synthesizable Verilog
  * Chisel one is used when result is `Bool` (i.e. result of Chisel comparison)
  * Can also customize assertion with failure message
* Scala `assert` checks during _construction_ but not _simulation_
  * Will be used if result is `Boolean` (i.e. result of Scala comparison)
* Recommend using `require` instead of Scala `assert`
  * Both `require` and `assert` built into Scala and evaluated at run time
  * Stylistically, `require` is for checking input sanity while `assert` is for checking internal consistency
    * Can also use flags to remove Scala `assert` (but not `require`) at compile time to reduce binary size

## Playing with Different Types of `assert`

In [27]:
class CheckNonZero(width: Int) extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(width.W))
        val out = Output(UInt(width.W))
    })
    require(width > 0)
    assert(io.in > 0.U, "saw >0 input")
    io.out := io.in
}

//printVerilog(new CheckNonZero(8))

defined class CheckNonZero

In [28]:
//printVerilog(new CheckNonZero(8))
println (getVerilog(new CheckNonZero(8)))

Elaborating design...
Done elaborating.
module CheckNonZero(
  input        clock,
  input        reset,
  input  [7:0] io_in,
  output [7:0] io_out
);
  assign io_out = io_in; // @[cmd26.sc 8:12]
  always @(posedge clock) begin
    `ifndef SYNTHESIS
    `ifdef PRINTF_COND
      if (`PRINTF_COND) begin
    `endif
        if (~(io_in > 8'h0 | reset)) begin
          $fwrite(32'h80000002,
            "Assertion failed: saw >0 input\n    at cmd26.sc:7 assert(io.in > 0.U, \"saw >0 input\")\n"); // @[cmd26.sc 7:11]
        end
    `ifdef PRINTF_COND
      end
    `endif
    `endif // SYNTHESIS
    `ifndef SYNTHESIS
    `ifdef STOP_COND
      if (`STOP_COND) begin
    `endif
        if (~(io_in > 8'h0 | reset)) begin
          $fatal; // @[cmd26.sc 7:11]
        end
    `ifdef STOP_COND
      end
    `endif
    `endif // SYNTHESIS
  end
endmodule



## Common Question: What Should Go in a Module vs Function vs Class?

* Do what makes it most clear to humans and easiest to reuse
    * Other apects are all secondary
* Defaulting to using just modules is just fine
    * Class that is not also a module is probably not commonly needed
* Things to consider what goes into an entity (module, function, etc...):
    * Design for _reuse_
    * Design for ease of _testing_ (with unit tests)
    * Design to _pull complexity downwards_
* Why ever not use a module?
    * Sometimes simpler/easier to make a function than making a full module including IO
    * Functions are easier to compose with a functional programming collective operation
    * If many instances, sometimes easier to work with (tool run time, Verilog hierarchy, waveforms) if lowest-level entities disolved as functions instead of modules


## "Flattening" & "Unflattening" Bundles

* Chisel generates a separate memory for each field of a `Bundle` (or element of a `Vec`)
  * This is usually what you want, so good default
  * However, sometimes, you may want to keep them all as one memory
* Can use `.getWidth` on a Bundle instance to see how many bits it is
* Can use `.asTypeOf` to cast bits into desired Bundle (sometimes called _reverse concatenation_)

In [29]:
class Pair extends Bundle {
    val a = UInt(1.W)
    val b = UInt(7.W)
}

class MemCohesion extends Module {
    val io = IO(new Bundle {
        val addr = Input(UInt(8.W))
        val out = Output(new Pair())
    })
    val m = Mem(256, new Pair)
    io.out := m(io.addr)
//     val m = Mem(256, UInt((new Pair).getWidth.W))
//     io.out := m(io.addr).asTypeOf(new Pair)
}

//printVerilog(new MemCohesion)

defined class Pair
defined class MemCohesion

In [30]:
//printVerilog(new MemCohesion)
println (getVerilog(new MemCohesion))

Elaborating design...
Done elaborating.
module MemCohesion(
  input        clock,
  input        reset,
  input  [7:0] io_addr,
  output       io_out_a,
  output [6:0] io_out_b
);
`ifdef RANDOMIZE_MEM_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
`endif // RANDOMIZE_MEM_INIT
  reg  m_a [0:255]; // @[cmd28.sc 11:16]
  wire  m_a_MPORT_data; // @[cmd28.sc 11:16]
  wire [7:0] m_a_MPORT_addr; // @[cmd28.sc 11:16]
  reg [6:0] m_b [0:255]; // @[cmd28.sc 11:16]
  wire [6:0] m_b_MPORT_data; // @[cmd28.sc 11:16]
  wire [7:0] m_b_MPORT_addr; // @[cmd28.sc 11:16]
  assign m_a_MPORT_addr = io_addr;
  assign m_a_MPORT_data = m_a[m_a_MPORT_addr]; // @[cmd28.sc 11:16]
  assign m_b_MPORT_addr = io_addr;
  assign m_b_MPORT_data = m_b[m_b_MPORT_addr]; // @[cmd28.sc 11:16]
  assign io_out_a = m_a_MPORT_data; // @[cmd28.sc 12:12]
  assign io_out_b = m_b_MPORT_data; // @[cmd28.sc 12:12]
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID